In [ ]:
import pandas as pd                     #Load libraries
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
dataset = pd.read_csv('movie_metadata.csv')
dataset.head()

In [ ]:
dataset.shape

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.groupby(by = 'color')['color'].count()

In [ ]:
dataset['color'] = dataset['color'].fillna('Color')

In [ ]:
dataset.groupby(by = 'color')['color'].count()

In [ ]:
duration = dataset[~dataset['duration'].isna()]

In [ ]:
duration

In [ ]:
duration_mean = dataset['duration'].mean()

In [ ]:
duration_mean

In [ ]:
duration['duration'].mean()

In [ ]:
dataset['duration'] = dataset['duration'].fillna(duration_mean)

In [ ]:
dataset['duration'].isnull().sum()

In [ ]:
dataset['num_critic_for_reviews'] = dataset['num_critic_for_reviews'].fillna(0)
dataset['director_facebook_likes'] = dataset['director_facebook_likes'].fillna(0) 
dataset['actor_1_facebook_likes'] = dataset['actor_1_facebook_likes'].fillna(0)
dataset['actor_2_facebook_likes'] = dataset['actor_2_facebook_likes'].fillna(0)
dataset['actor_3_facebook_likes'] = dataset['actor_3_facebook_likes'].fillna(0)
dataset['facenumber_in_poster'] = dataset['facenumber_in_poster'].fillna(0)
dataset['num_user_for_reviews'] = dataset['num_user_for_reviews'].fillna(0)

In [ ]:
dataset.groupby('language')['language'].count()

In [ ]:
dataset['language'] = dataset['language'].fillna('English')

In [ ]:
dataset['country'] = dataset['country'].fillna('USA')

In [ ]:
import requests, re
def link_replacer(link):             
    link = link.replace('http://www.movie.com/title/', 'https://www.imdb.com/title/')
    link = link.replace('/?ref_=fn_tt_tt_1', '/')
    return link


def budget_extractor(link):           
    
    print(link)
    r = requests.get(link)
    if re.search('<h4 class="inline">Budget:</h4>(.*?)<span class="attribute">',r.text, re.DOTALL) is None:
        return np.NAN
    content = re.search('<h4 class="inline">Budget:</h4>(.*?)<span class="attribute">',r.text,re.DOTALL).group(1)
    content = re.sub('\W+','', content)
    content = re.search(r'\d+', content).group()
    content = int(content)
    return content


def gross_extractor(link):         
    print(link)
    r = requests.get(link)
    if re.search('<h4 class="inline">Gross USA:</h4>(.*?)</div>',r.text, re.DOTALL) is None:
                                                    #Check if Gross USA values exists
        if re.search('<h4 class="inline">Cumulative Worldwide Gross:</h4>(.*?)</div>',r.text, re.DOTALL) is None:
            return np.NAN                       #Check if Cummulative Gross values exists
        else:
            content = re.search('<h4 class="inline">Cumulative Worldwide Gross:</h4>(.*?)</div>',r.text,re.DOTALL).group(1)
            content = re.sub('\W+','', content)
            content = re.search(r'\d+', content).group()
            content = int(content)
            return content
    else:
        content = re.search('<h4 class="inline">Gross USA:</h4>(.*?)</div>',r.text,re.DOTALL).group(1)
        content = re.sub('\W+','', content)
        content = re.search(r'\d+', content).group()
        content = int(content)
        return content
dataset['movie_imdb_link'] = dataset['movie_imdb_link'].apply(lambda x: link_replacer(x))


dataset['gross'] = dataset['movie_imdb_link'].apply(lambda x: gross_extractor(x))



dataset['budget'] = dataset['movie_imdb_link'].apply(lambda x: budget_extractor(x))


dataset.isna().sum()

In [ ]:
dataset = dataset[~dataset['title_year'].isna()]

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.drop(['director_name','actor_2_name','actor_1_name','movie_title','actor_3_name','plot_keywords','movie_imdb_link','title_year','aspect_ratio'], axis =1, inplace = True)

In [ ]:
dataset.head()

In [ ]:
top3_content_rating = dataset["content_rating"].value_counts()[:3].index
dataset['content_rating'] = dataset.content_rating.where(dataset.content_rating.isin(top3_content_rating), 'other')
dataset = pd.get_dummies(dataset, columns=['content_rating'], prefix = ['content_rating'])

In [ ]:
top3_country = dataset["country"].value_counts()[:3].index
dataset['country'] = dataset.country.where(dataset.country.isin(top3_country), 'other')
dataset = pd.get_dummies(dataset, columns=['country'], prefix = ['country'])

In [ ]:
dataset.head()

In [ ]:
top3_language = dataset["language"].value_counts()[:3].index
dataset['language'] = dataset.language.where(dataset.language.isin(top3_language), 'other')
dataset = pd.get_dummies(dataset, columns=['language'], prefix = ['language'])

In [ ]:
dataset.head()

In [ ]:
dataset = pd.get_dummies(dataset, columns=['color'],prefix=['color'])

In [ ]:
dataset.head()

In [ ]:
dataset.describe()

In [ ]:
dataset.info()

In [ ]:
corr_matrix = dataset.corr()
plt.figure(figsize = (12,12))
sns.heatmap(corr_matrix, annot = True)
plt.show()

In [ ]:
dataset['actors_facebook_likes']=dataset['actor_1_facebook_likes']+dataset['actor_2_facebook_likes']+dataset['actor_3_facebook_likes']
dataset=dataset.drop(columns=['actor_1_facebook_likes','actor_2_facebook_likes','actor_3_facebook_likes'])

In [ ]:
corr_matrix = dataset.corr()
plt.figure(figsize = (12,12))
sns.heatmap(corr_matrix, annot = True)
plt.show()

In [ ]:
dataset.drop('actors_facebook_likes',axis =1, inplace = True)

In [ ]:
corr_matrix = dataset.corr()
plt.figure(figsize = (12,12))
sns.heatmap(corr_matrix, annot = True)
plt.show()

In [ ]:
dataset.info()

In [ ]:
dataset.drop_duplicates(inplace = True)

In [ ]:
dataset.dropna(inplace = True)

In [ ]:
dataset.shape

In [ ]:
dataset.info()

In [ ]:
dataset.drop('genres', axis = 1, inplace = True)

In [ ]:
X = dataset.drop(columns=['imdb_score'])
X
y = dataset['imdb_score']
y
X = np.array(X)
y = np.array(y)
y.shape
#from sklearn.preprocessing import StandardScaler, MinMaxScaler
#scaler_x = StandardScaler()
#X = scaler_x.fit_transform(X)
#scaler_y = StandardScaler()
#y = scaler_x.fit_transform(y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)
from sklearn.ensemble import RandomForestRegressor
RandomForest_model = RandomForestRegressor(n_estimators = 200)
RandomForest_model.fit(X_train, y_train)
accuracy_RandomForest = RandomForest_model.score(X_test, y_test)
accuracy_RandomForest

In [ ]:
y_pred=RandomForest_model.predict(X_test)
print("Final rmse value is =",np.sqrt(np.mean((y_test-y_pred)**2)))